In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.set() # matplot lib defaults
plt.rcParams['figure.figsize'] = (16, 8)
%config InlineBackend.figure_format='retina'

In [ ]:
# find the notebook the saved figures came from
fig_prefix='../figures/jq-'

In [ ]:
# from IPython.display import IFrame
# IFrame('https://dataviz.vam.wfp.org/economic_explorer/price-forecasts-alerts?adm0=205', width=700, height=400)

In [ ]:
df =pd.read_csv('../data/eac-ratin.csv')
df.tail(5)

In [ ]:
# import qgrid
# qgrid.nbinstall(overwrite=True)
# qgrid.show_grid(df, remote_js=True)

Data cleaning: NaN, dtype conversion

In [ ]:
df = df.rename(columns = lambda x: x.lower())
cols = df.columns.tolist()
df = df.rename(columns={cols[-3]: 'retail', cols[-2]: 'wholesale'})
df.columns.tolist()

In [ ]:
df.dtypes

In [ ]:
df['wholesale'].sort_values().unique()

In [ ]:
df[df['wholesale']=='Wholesale'] = np.NaN
df['wholesale']=df['wholesale'].astype('float')

In [ ]:
df['retail'].sort_values().unique()

In [ ]:
df[df['retail']=='NaN'] = np.NaN
df['retail'] = df['retail'].astype('float')
df['retail'].sort_values().unique()

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df.dtypes

In [ ]:
str_cols = ['market', 'product', 'country', 'currency']
df[str_cols].describe()

for item in str_cols:
    df[item]=df[item].astype('category') # which will by default set the length to the max len it encounters
    print(df[item].unique())

In [ ]:
df.dtypes

In [ ]:
df['year']=df['date'].dt.year
df['month']=df['date'].dt.month
df['day']=df['date'].dt.day
df['week']=df['date'].dt.week
# note that since there are nan in the date colunms, let the dtype remain float

In [ ]:
df.head(3)

In [ ]:
np.nan


In [ ]:
# replace zeros with NaN

cols = ['wholesale', 'retail']
df[cols] = df[cols].replace({0:np.nan})

# cond = (df['wholesale']==0)
# df_zero = df[cond]
# df_zero['wholesale'] = np.NaN
# df['wholesale']=df_zero['wholesale']


In [ ]:
# return 1 to confirm no zero values
np.prod(df['wholesale']!=0)

In [ ]:
# remove outliers:
def remove_outliers(x):
    '''x: pandas series'''
    lower_bound, upper_bound = x.quantile(.05), x.quantile(.95)
    x = x[x.between(lower_bound, upper_bound)]
    return x

df['wholesale'] = remove_outliers(df['wholesale'])
df['retail'] = remove_outliers(df['retail'])

exploratory visualization:

In [ ]:
# save a copy of cleaned data to csv
df.to_csv('../data/cleaned_data.csv', index=False)

In [ ]:
# save to db file
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../data/mydb.db', echo=False)
df.to_sql('data', con=engine, if_exists='replace',
           index_label='id')